<a href="https://colab.research.google.com/github/christohmg/Image-Resolution-with-Autoencoders-/blob/main/Image_Resolution_with_Autoencoders_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
import random
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, add
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Function to load and resize images from a specified local folder
def load_and_resize_images(folder, target_size, resize_factor):
    images, small_images = [], []
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            img_path = os.path.join(folder, filename)
            with Image.open(img_path) as img:
                img_resized = img.resize((target_size, target_size))
                images.append(np.array(img_resized))

                small_size = int(target_size * resize_factor)
                img_small = img.resize((small_size, small_size))
                small_images.append(np.array(img_small))

    return np.array(images) / 255.0, np.array(small_images) / 255.0

# Paths to your datasets, adjust as needed
train_folder = 'data/train' # Adjust to your path
test_folder = 'data/test' # Adjust to your path
target_size = 256
resize_factor = 0.25

# Load and process images
train_images, processed_train_images = load_and_resize_images(train_folder, target_size, resize_factor)
test_images, processed_test_images = load_and_resize_images(test_folder, target_size, resize_factor)

In [ ]:
index_to_visualize = random.randint(0, len(train_images) - 1)

# Visualize the original and processed images side by side
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Original Image
axes[0].imshow(train_images[index_to_visualize])
axes[0].title.set_text("Original Image")

# Processed Image
axes[1].imshow(processed_train_images[index_to_visualize])
axes[1].title.set_text("Processed Image")

plt.show()

In [ ]:
input_img = Input(shape=(64, 64, 3))

# Encoder
l1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
l2 = Conv2D(32, (3, 3), activation='relu', padding='same')(l1)
l3 = MaxPooling2D((2, 2), padding='same')(l2)
l3 = Dropout(0.3)(l3)
l4 = Conv2D(64, (3, 3), activation='relu', padding='same')(l3)
l5 = Conv2D(64, (3, 3), activation='relu', padding='same')(l4)
l6 = MaxPooling2D((2, 2), padding='same')(l5)
l7 = Conv2D(128, (3, 3), activation='relu', padding='same')(l6)

# Decoder
l8 = UpSampling2D((2, 2))(l7)
l9 = Conv2D(64, (3, 3), activation='relu', padding='same')(l8)
l10 = Conv2D(64, (3, 3), activation='relu', padding='same')(l9)
# Skip connection, adjusted due to different layer sizes
l11 = add([l5, l10])
l12 = UpSampling2D((2, 2))(l11)
l13 = Conv2D(32, (3, 3), activation='relu', padding='same')(l12)
l14 = Conv2D(32, (3, 3), activation='relu', padding='same')(l13)
# Skip connection, adjusted due to different layer sizes
l15 = add([l14, l2])

# Output layer
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(l15)

# Compile the model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Display the summary of the model
autoencoder.summary()

In [ ]:
model_path = "autoencoder.h5"
checkpoint = ModelCheckpoint(model_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=9,
                          verbose=1,
                          restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=5,
                                            verbose=1,
                                            factor=0.2,
                                            min_lr=0.00000001)

hist = autoencoder.fit(
    processed_train_images,  # Input images
    processed_train_images,  # Target images are the same as input
    epochs=10,
    batch_size=16,
    shuffle=True,
    validation_data=(processed_test_images, processed_test_images),  # Validation data
    callbacks=[checkpoint, earlystop, learning_rate_reduction]  # Include your callbacks here
)

In [ ]:
# Enhance a low-resolution image from the test dataset
image_index = random.randint(0, len(processed_test_images) - 1)
low_res_image = processed_test_images[image_index]
enhanced_image = autoencoder.predict(np.expand_dims(low_res_image, axis=0))[0]

# Visualization of the original, low-resolution, and enhanced images
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(test_images[image_index])
axes[0].set_title("Original Image")
axes[1].imshow(low_res_image)
axes[1].set_title("Low-Resolution Image")
axes[2].imshow(enhanced_image)
axes[2].set_title("Enhanced Image")
for ax in axes:
    ax.axis('off')
plt.show()